[View in Colaboratory](https://colab.research.google.com/github/Dineshchitta/DeepLearning-Practice/blob/master/Naval_mine_identifier.ipynb)

**Source:**

The data set was contributed to the benchmark collection by Terry Sejnowski, now at the Salk Institute and the University of California at San Deigo. The data set was developed in collaboration with R. Paul Gorman of Allied-Signal Aerospace Technology Center.

**Data Set Information:**

The file "sonar.mines" contains 111 patterns obtained by bouncing sonar signals off a metal cylinder at various angles and under various conditions. The file "sonar.rocks" contains 97 patterns obtained from rocks under similar conditions. The transmitted sonar signal is a frequency-modulated chirp, rising in frequency. The data set contains signals obtained from a variety of different aspect angles, spanning 90 degrees for the cylinder and 180 degrees for the rock.

Each pattern is a set of 60 numbers in the range 0.0 to 1.0. Each number represents the energy within a particular frequency band, integrated over a certain period of time. The integration aperture for higher frequencies occur later in time, since these frequencies are transmitted later during the chirp.

The label associated with each record contains the letter "R" if the object is a rock and "M" if it is a mine (metal cylinder). The numbers in the labels are in increasing order of aspect angle, but they do not encode the angle directly.

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 

In [0]:
#Loading the data
data = pd.read_csv('https://raw.githubusercontent.com/malinenimaurya/datasets/master/Sonar.csv')

In [5]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V52,V53,V54,V55,V56,V57,V58,V59,V60,Class
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,1
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,1
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,1
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,1
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,1


In [21]:
data[data.columns[60]].value_counts()

0    111
1     97
Name: Class, dtype: int64

 **Input and Output**

In [0]:
X = data[data.columns[0:60]].values
y = data[data.columns[60]].values

In [8]:
print(x.shape)

(208, 60)


In [0]:
#One hot encoding the target variable
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [0]:
Y = one_hot_encode(y)

In [0]:
#Need to shuffle as the all Rocks and Mines are together
X,Y = shuffle (X, Y, random_state = 0)

In [26]:
# Splittting the data into train and test
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size = 0.20, random_state = 0)
print(train_x)

[[0.034  0.0625 0.0381 ... 0.0051 0.0033 0.0058]
 [0.0253 0.0808 0.0507 ... 0.0039 0.0081 0.0053]
 [0.0368 0.0279 0.0103 ... 0.0086 0.011  0.0052]
 ...
 [0.0291 0.04   0.0771 ... 0.0081 0.0139 0.0111]
 [0.0257 0.0447 0.0388 ... 0.0199 0.0255 0.018 ]
 [0.0137 0.0297 0.0116 ... 0.0035 0.0043 0.0033]]


In [0]:
#Code for the Artificial neural net

In [38]:
IN = tf.placeholder(tf.float32, [None, 60])
W1 = tf.Variable(tf.truncated_normal([60, 2], stddev=0.1)) #random initialisations of weights
b1 = tf.Variable(tf.ones([2])/2)  #small positive initialisation for bias

Ylogits = tf.matmul(IN,W1) + b1
# SOFT_MAX = tf.nn.softmax(Ylogits)

Y_ = tf.placeholder(tf.float32, [None, 2])

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_) # handled nan(log0) for cross entropy, different optimisers
cross_entropy = tf.reduce_mean(cross_entropy)*100

is_correct = tf.equal(tf.argmax(Ylogits,1), tf.argmax(Y_,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

optimizer = tf.train.AdamOptimizer(0.003)         #can use RMSprop, Adadelta, gradientdescent etc

train_step = optimizer.minimize(cross_entropy)

 


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



ValueError: ignored

In [42]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
accuracy_history = []
for i in range(1000):
    # load batch of images and correct answers
#     batch_X, batch_Y = data.train.next_batch(100)
    train_data={IN: train_x, Y_: train_y}

    # train
    sess.run(train_step, feed_dict=train_data)
    if i%50 == 0:
      a,c = sess.run([accuracy, cross_entropy], feed_dict=train_data)
      accuracy_history.append(a)
      print("iteration:"+ str(i) + ", accuracy:" + str(a) +", cross_entropy:"+ str(c))

iteration:0, accuracy:0.56626505, cross_entropy:69.00089
iteration:50, accuracy:0.6385542, cross_entropy:66.064
iteration:100, accuracy:0.6506024, cross_entropy:64.75083
iteration:150, accuracy:0.6506024, cross_entropy:63.770348
iteration:200, accuracy:0.6506024, cross_entropy:62.952168
iteration:250, accuracy:0.64457834, cross_entropy:62.25037
iteration:300, accuracy:0.6566265, cross_entropy:61.639584
iteration:350, accuracy:0.64457834, cross_entropy:61.10124
iteration:400, accuracy:0.6506024, cross_entropy:60.620934
iteration:450, accuracy:0.6566265, cross_entropy:60.187485
iteration:500, accuracy:0.6626506, cross_entropy:59.79234
iteration:550, accuracy:0.67469877, cross_entropy:59.42908
iteration:600, accuracy:0.6686747, cross_entropy:59.092827
iteration:650, accuracy:0.6807229, cross_entropy:58.779854
iteration:700, accuracy:0.6927711, cross_entropy:58.487278
iteration:750, accuracy:0.6927711, cross_entropy:58.21279
iteration:800, accuracy:0.6927711, cross_entropy:57.954563
iterat